In [1]:
from funcs_general import *
sys.path.insert(0,'/var/www/vhosts/algotrade.glueckert.net/projects/') 
from SETTINGS.settings_auth import *
from funcs_traindata import *

# more data: http://www.cryptodatasets.com/

In [3]:
def exit_handler():
    if  'pool' in globals() or 'pool' in locals():   pool.terminate()
    if  'WPOOL' in globals() or 'WPOOL' in locals():   WPOOL.terminate()
    print('App terminated, closing Pool.')

def make_data(e):
    symbol = e['symbol']
    dt = e['date']
    min_ts = e['start']
    max_ts = e['end']
    print('-------------creating features day...: ' + str(dt))
    data = pd.DataFrame({})
    try:
        data = make_features(symbol,
                             min_ts=-2000+min_ts,
                             max_ts=2000+max_ts,
                             mid_offsets=[60,120,300,600,900,1800],
                             trades_offsets=[30, 60, 120,360,900],
                             powers=[2, 4])
        data = data[~data.index.duplicated()].sort_index()
        dates = [datetime.datetime.fromtimestamp(int(d)).date() for d in data.index]
        data = data.loc[np.isin(dates,dt),:]
        print("data n : " + str(data.shape[0]))
        data.to_csv('bitfinex_btcusd/'+ symbol + '_' + str(dt)  + '.csv')
    except Exception as e:
        with open('bitfinex_btcusd/log_' + symbol + '_' + str(dt)  + '.txt', 'w') as f: f.write(str(e))
    gc.collect()
        


In [4]:
from multiprocessing import Pool

WPOOL = Pool(12)

if __name__ == '__main__':
    WPOOL.map(make_data, get_books_days_ts(symbol='btc',date_from=date(2017,12,30),date_to=date(2017,12,31)))
    WPOOL.close()
    d = merge_csvs(path = 'bitfinex_btcusd/')
    d.to_csv('training_data/bitfinex_btcusd.csv.gz', index=False,header=True,compression='gzip')

-------------creating features day...: 2017-12-30
-------------creating features day...: 2017-12-31


/apps/anaconda3/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "
/apps/anaconda3/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "


get book data run time: 0.6241733312606812 minutes
get book data run time: 0.6258537769317627 minutes
width and mid run time: 0.10150318543116252 minutes
width and mid run time: 0.10652031898498535 minutes
offset mids run time: 0.6386520663897196 minutes
offset mids run time: 0.6435851136843364 minutes
power calcs run time: 4.184088687102 minutes
power calcs run time: 4.2201513449351 minutes
trade features run time: 6.812622809410096 minutes
make_features run time: 12.362808835506438 minutes
trade features run time: 7.066309603055318 minutes
make_features run time: 12.660826472441356 minutes
